<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Домашнее-задание-по-теме-&quot;Коническая-оптимизация-для-управления-портфелем&quot;" data-toc-modified-id="Домашнее-задание-по-теме-&quot;Коническая-оптимизация-для-управления-портфелем&quot;-1">Домашнее задание по теме "Коническая оптимизация для управления портфелем"</a></span></li><li><span><a href="#Задача-4" data-toc-modified-id="Задача-4-2">Задача 4</a></span></li><li><span><a href="#Задача-5" data-toc-modified-id="Задача-5-3">Задача 5</a></span></li><li><span><a href="#Задача-6" data-toc-modified-id="Задача-6-4">Задача 6</a></span></li><li><span><a href="#Задача-7-(сложная)" data-toc-modified-id="Задача-7-(сложная)-5">Задача 7 (сложная)</a></span></li></ul></div>

In [84]:
from math import sqrt
import numpy as np
from cvxopt import matrix
from cvxopt.blas import nrm2, dot
from cvxopt.solvers import qp, socp, options
import matplotlib.pyplot as plt
%matplotlib inline

# Домашнее задание по теме "Коническая оптимизация для управления портфелем"
для получение полного бала необходимо решить задачу 4 и одну из задач 5 или 6.

Задача 7 - задача для сомостоятельной работы для тех кто хочет больше углубиться в тему. Любые мысли на тему ее решения приветствуются, но в зачет по домашнему заданию не входят.

# Задача 4
Напишите функцию, которая решает задачу выбора оптимального портфеля Марковица по следующим входным данным: 

- матрица $A$ разложения инструментов по факторам: один размер $N$ определяет число инструментов, второй размер $K$ определяет число факторов;
- матрица $B$ ковариации факторов (порядок факторов считать согласованным с матрицей $A$;
- вектор $D$ длины $N$ идиосинкратического риска: компонента $i$ вектора равна $Var(\delta_i)$;
- вектор $\alpha$ ожидаемых доходностей инструментов;
- параметр "неприятия риска" $\lambda$;
- максимальная сумма всех позиций портфеля по абсолютной величине (maxGross); 
- флаг, разрешающий или запрещающий короткие (отрицательные) позиции инструментов в портфеле;
- в случае если короткие позиции разрешены, минимальная/максимальная сумма всех позиций портфеля (minNet / maxNet);

Требования к выходным данным: 

- позиции всех инструментов в портфеле; 
- позиции портфеля по всем факторам;
- риск портфеля и ожидаемую доходность.

Было бы очень хорошо аккуратно обрабатывать случаи, когда оптимизатор говорит, что проблема либо не имеет решения, либо выдет статус решения, отличный от Optimal solution found. 

Сигнатура функции:

In [127]:
def problem4(A, B, D, alpha, riskAversion, maxGross, allowShort=False, maxNet=None, minNet=None):
    alpha=matrix(alpha)
    K=A.shape[0]
    N=A.shape[1]
    Cov = A.T @ B @ A + D
    rootCov = np.linalg.cholesky(Cov).T
    rootCov = matrix(rootCov)
    if allowShort==False:
        coeffUtil = matrix([riskAversion, -1.0*alpha])

        # investment limit plus positivity constraints
        G0 = matrix([matrix([[0.0],[matrix(1.0, (1,N))]]),
                     -1.0*matrix(np.eye(N+1))])
        h0 = matrix([[maxGross, matrix(0.0, (N+1,1))]])

        # conic constraint
        z4 = matrix(0.0, (1,N))
        G1 = matrix([[-1, z4.T],[z4, -1.0*rootCov]])
        h1 = matrix(0.0, (N+1,1))    

        # linear constraint = full investment
        F = matrix([[0.0], [matrix(1.0, (1,N))]])
        b = matrix(float(maxGross), (1,1))
        
        solution = socp(coeffUtil, G0, h0, [G1], [h1], F, b)
        
    else:
        h1 = matrix(0.0, (2*N+1,1))
        G1 = matrix([[-1, matrix(0.0,(2*N,1))],[matrix(0.0,(1,N)), 
                                                -1.0*rootCov, matrix(0.0,(N,N))],[matrix(0.0,(2*N+1,N))]])        
        if (maxNet!=None) & (minNet==None) :
            m_z=matrix(0.0,(N,1))
            coeffUtil = matrix([riskAversion, -1.0*alpha,m_z])
            h0=matrix([0.0,maxGross,maxNet,matrix(0.0,(2*N,1))])
            rm=matrix([[-1.0],[matrix(0.0,(1,2*N))]])
            mgr=matrix([[matrix(0.0,(1,N+1))],[matrix(1.0,(1,N))]])
            maxn=matrix([[0.0],[matrix(1.0,(1,N))],[matrix(0.0,(1,N))]])

            usl1=matrix([[matrix(0.0,(N,1))],[matrix(np.eye(N))],[-1.0*matrix(np.eye(N))]])
            usl2=matrix([[matrix(0.0,(N,1))],[-1.0*matrix(np.eye(N))],[-1.0*matrix(np.eye(N))]])

            G0=matrix([rm,mgr,maxn,usl1,usl2])
            
            
        elif (maxNet==None) & (minNet!=None):
            m_z=matrix(0.0,(N,1))
            coeffUtil = matrix([riskAversion, -1.0*alpha,m_z])

            h0=matrix([0.0,maxGross,-minNet,matrix(0.0,(2*N,1))])

            rm=matrix([[-1.0],[matrix(0.0,(1,2*N))]])
            mgr=matrix([[matrix(0.0,(1,N+1))],[matrix(1.0,(1,N))]])
            minn=-1.0*matrix([[0.0],[matrix(1.0,(1,N))],[matrix(0.0,(1,N))]])

            usl1=matrix([[matrix(0.0,(N,1))],[matrix(np.eye(N))],[-1.0*matrix(np.eye(N))]])
            usl2=matrix([[matrix(0.0,(N,1))],[-1.0*matrix(np.eye(N))],[-1.0*matrix(np.eye(N))]])

            G0=matrix([rm,mgr,maxn,minn,usl1,usl2])
        
        else:
            m_z=matrix(0.0,(N,1))
            coeffUtil = matrix([riskAversion, -1.0*alpha,m_z])

            h0=matrix([0.0,maxGross,maxNet,-minNet,matrix(0.0,(2*N,1))])

            rm=matrix([[-1.0],[matrix(0.0,(1,2*N))]])
            mgr=matrix([[matrix(0.0,(1,N+1))],[matrix(1.0,(1,N))]])
            maxn=matrix([[0.0],[matrix(1.0,(1,N))],[matrix(0.0,(1,N))]])
            minn=-1.0*maxn

            usl1=matrix([[matrix(0.0,(N,1))],[matrix(np.eye(N))],[-1.0*matrix(np.eye(N))]])
            usl2=matrix([[matrix(0.0,(N,1))],[-1.0*matrix(np.eye(N))],[-1.0*matrix(np.eye(N))]])

            G0=matrix([rm,mgr,maxn,minn,usl1,usl2])
        
        solution = socp(coeffUtil, G0, h0, [G1], [h1])

    
    pi = solution['x'][1:N+1]
    profit = dot(alpha,pi)
    Cov=matrix(Cov)
    risk = sqrt(dot(pi, Cov*pi))
    pi_factors = [dot(pi,matrix(A[i])) for i in range(A.shape[0])]

    return pi, pi_factors, risk, profit

Когда решение готово запусите код в следующей ячейке, он распечатает результаты для некоторых вариантов расчета с тестовыми данными.

In [132]:
from test_data import factor_cov, stocks_to_factors, stock_specific_risk, stock_alpha, run_problem4_tests

A = stocks_to_factors.to_numpy().T
B = factor_cov.to_numpy()
D = np.diag( np.ravel( stock_specific_risk ) )
alpha = np.ravel( stock_alpha )

run_problem4_tests(solution=problem4, A=A, B=B, D=D, alpha=alpha)

{'riskAversion': 0.0, 'maxGross': 1000000, 'allowShort': False}
sum: 999999.9999999999; profit: 149999.99667391815; risk: 140770.1754410287
[ 1.46095004e-03  5.15969429e-03  5.63649803e-03  5.79347553e-03
  9.99999959e+05 -1.39937846e-04  5.73010020e-03  5.93499509e-03
  5.72735249e-03  5.61002974e-03]
[ 1.00000000e+06 -6.99999993e+05  1.50000015e+05  8.49999985e+05
  9.99999984e+05  1.64062221e-02]

{'riskAversion': 0.5, 'maxGross': 1000000, 'allowShort': False}
sum: 1000000.0000000006; profit: 149999.99915837916; risk: 140770.17590431916
[-8.82744524e-07  1.58309145e-03  2.03359767e-03  1.17034117e-03
  9.99999990e+05  1.01022848e-04  1.03072812e-03  1.08023520e-03
  1.16339788e-03  2.02005006e-03]
[ 1.00000000e+06 -6.99999997e+05  1.50000004e+05  8.49999996e+05
  9.99999994e+05  5.63673918e-03]

{'riskAversion': 1.0, 'maxGross': 1000000, 'allowShort': False}
sum: 1000000.0000000001; profit: 149999.9997124758; risk: 140770.17606431953
[3.55192063e-04 4.20187039e-04 7.88070259e-04 3.0

# Задача 5
Добавьте в задачу оптимизации 4 линейные органичения по факторам. К входным данным добавляется список индексов факторов, для которых будут ограничения. Каждое ограничение представляет собой пару чисел: максимальное и минимальное значение суммарной позиции портфеля по фактору. 

In [139]:
def problem5(A, B, D, alpha, riskAversion, maxGross,list, allowShort=False, maxNet=None, minNet=None):
    #Cчитаем что list выглядит так:[[id1,max1,min1],[id2,max2,min2]......]. Тогда допишем в G0 нужные строчки матрицы
    #A дважды (соответствующие id_i), а в h0 допишем cначала все max_i из листа, потом все min_i из листа
    alpha=matrix(alpha)
    K=A.shape[0]
    N=A.shape[1]
    Cov = A.T @ B @ A + D
    rootCov = np.linalg.cholesky(Cov).T
    rootCov = matrix(rootCov)
    if allowShort==False:
        coeffUtil = matrix([riskAversion, -1.0*alpha])
        
        a_c=matrix(0.0, (len(list),N))
        a_c=np.array(a_c)
        for i in range(len(list)):
            a_c[i]=A[list[i][0]]
        a_c=matrix(a_c)
        g=matrix([[matrix(0.0,(len(list),1))],[a_c]])
        G0 = matrix([matrix([[0.0],[matrix(1.0, (1,N))]]),
                     -1.0*matrix(np.eye(N+1)),g,-g])
        
        h0 = matrix([[maxGross, matrix(0.0, (N+1,1))]])
        for i in range(len(list)):
            h0=matrix([h0,list[i][1]])
        for i in range(len(list)):
            h0=matrix([h0,-list[i][2]])

        # conic constraint
        z4 = matrix(0.0, (1,N))
        G1 = matrix([[-1, z4.T],[z4, -1.0*rootCov]])
        h1 = matrix(0.0, (N+1,1))    

        # linear constraint = full investment
        F = matrix([[0.0], [matrix(1.0, (1,N))]])
        b = matrix(float(maxGross), (1,1))

        solution = socp(coeffUtil, G0, h0, [G1], [h1], F, b)
        
    else:
        h1 = matrix(0.0, (2*N+1,1))
        G1 = matrix([[-1, matrix(0.0,(2*N,1))],[matrix(0.0,(1,N)), 
                                                -1.0*rootCov, matrix(0.0,(N,N))],[matrix(0.0,(2*N+1,N))]])        
        if (maxNet!=None) & (minNet==None) :
            m_z=matrix(0.0,(N,1))
            coeffUtil = matrix([riskAversion, -1.0*alpha,m_z])
            h0=matrix([0.0,maxGross,maxNet,matrix(0.0,(2*N,1))])
            for i in range(len(list)):
                h0=matrix([h0,list[i][1]])
            for i in range(len(list)):
                h0=matrix([h0,-list[i][2]])
                     
            rm=matrix([[-1.0],[matrix(0.0,(1,2*N))]])
            mgr=matrix([[matrix(0.0,(1,N+1))],[matrix(1.0,(1,N))]])
            maxn=matrix([[0.0],[matrix(1.0,(1,N))],[matrix(0.0,(1,N))]])

            usl1=matrix([[matrix(0.0,(N,1))],[matrix(np.eye(N))],[-1.0*matrix(np.eye(N))]])
            usl2=matrix([[matrix(0.0,(N,1))],[-1.0*matrix(np.eye(N))],[-1.0*matrix(np.eye(N))]])

            a_c=matrix(0.0, (len(list),N))
            a_c=np.array(a_c)
            for i in range(len(list)):
                a_c[i]=A[list[i][0]]
            a_c=matrix(a_c)
            a_c=matrix([[a_c],[matrix(0.0,(len(list),N))]])
            g=matrix([[matrix(0.0,(len(list),1))],[a_c]])
                                         
            G0=matrix([rm,mgr,maxn,usl1,usl2,g,-g])
            
        elif (maxNet==None) & (minNet!=None):
            m_z=matrix(0.0,(N,1))
            coeffUtil = matrix([riskAversion, -1.0*alpha,m_z])

            h0=matrix([0.0,maxGross,-minNet,matrix(0.0,(2*N,1))])
            
            for i in range(len(list)):
                h0=matrix([[h0,list[i][1]]])
            for i in range(len(list)):
                h0=matrix([[h0,-list[i][2]]])
    
            
            rm=matrix([[-1.0],[matrix(0.0,(1,2*N))]])
            mgr=matrix([[matrix(0.0,(1,N+1))],[matrix(1.0,(1,N))]])
            minn=-1.0*matrix([[0.0],[matrix(1.0,(1,N))],[matrix(0.0,(1,N))]])

            usl1=matrix([[matrix(0.0,(N,1))],[matrix(np.eye(N))],[-1.0*matrix(np.eye(N))]])
            usl2=matrix([[matrix(0.0,(N,1))],[-1.0*matrix(np.eye(N))],[-1.0*matrix(np.eye(N))]])

            a_c=matrix(0.0, (len(list),N))
            a_c=np.array(a_c)
            for i in range(len(list)):
                a_c[i]=A[list[i][0]]
            a_c=matrix(a_c)
            a_c=matrix([[a_c],[matrix(0.0,(len(list),N))]])
            g=matrix([[matrix(0.0,(len(list),1))],[a_c]])
                                         
            G0=matrix([rm,mgr,maxn,usl1,usl2,g,-g])
            G0=matrix([rm,mgr,maxn,minn,usl1,usl2])
        
        else:
            m_z=matrix(0.0,(N,1))
            coeffUtil = matrix([riskAversion, -1.0*alpha,m_z])

            h0=matrix([0.0,maxGross,maxNet,-minNet,matrix(0.0,(2*N,1))])
            for i in range(len(list)):
                h0=matrix([h0,list[i][1]])
            for i in range(len(list)):
                h0=matrix([h0,-list[i][2]])
            
            rm=matrix([[-1.0],[matrix(0.0,(1,2*N))]])
            mgr=matrix([[matrix(0.0,(1,N+1))],[matrix(1.0,(1,N))]])
            maxn=matrix([[0.0],[matrix(1.0,(1,N))],[matrix(0.0,(1,N))]])
            minn=-1.0*maxn

            usl1=matrix([[matrix(0.0,(N,1))],[matrix(np.eye(N))],[-1.0*matrix(np.eye(N))]])
            usl2=matrix([[matrix(0.0,(N,1))],[-1.0*matrix(np.eye(N))],[-1.0*matrix(np.eye(N))]])

            a_c=matrix(0.0, (len(list),N))
            a_c=np.array(a_c)
            for i in range(len(list)):
                a_c[i]=A[list[i][0]]
            a_c=matrix(a_c)
            a_c=matrix([[a_c],[matrix(0.0,(len(list),N))]])
            g=matrix([[matrix(0.0,(len(list),1))],[a_c]])
                                         
            G0=matrix([rm,mgr,maxn,usl1,usl2,g,-g])
            
        solution = socp(coeffUtil, G0, h0, [G1], [h1])

    
    pi = solution['x'][1:N+1]
    profit = dot(alpha,pi)
    Cov=matrix(Cov)
    risk = sqrt(dot(pi, Cov*pi))
    pi_factors = [dot(pi,matrix(A[i])) for i in range(A.shape[0])]

    return pi, pi_factors, risk, profit

# Задача 6
Модифицируйте функцию из задачи 3 или задачи 4 для решения задачи ребалансировки портфеля (см. слайд "Оптимальная ребалансировка портфеля" лекции). Имеется начальный портфель $w$, который требуется оптимально изменить путем добавления "вектора ребалансировки" $x$. Компоненты этого вектора могут быть как положительны (докупаем акций), так и отрицательными (продаем акции), но результирующий портфель $w+x$ должен иметь все положительные компоненты, причем 

$\sum_{i=1}^{N} w_i = \sum_{i=1} (w_i + x_i)$.

В модифицированной функции оптимизации Марковица добавляется сумма транзакционных издержек $TC(x)$. Издержки торговли одним инструментом не влияют на другие инструменты, они пропорциональны количеству проторгованного:

$TC(x) = \sum_{i=1}^{N} p_i |x_i|$.

Коэффициенты пропорциональности считаем постоянными величинами, зависящими отсвойств акции. Вектор этих коэффициентов добавляется в спсиок входных данных.

# Задача 7 (сложная)
Написать функцию, реализующую оптимальную ликвидацию портфеля  в несколько шагов. Имеется начальный портфель $w$, от которого нужно полностью избавиться за несколько шагов, то есть построить кусочно-линейную траекторию $w(t)$ с узлами $w_0= w$, $w_1, \ldots w_{k-1}, w_k = 0$. Число шагов $k$ известно заранее, это параметр задачи. Траектория ликвидации портфеля $w(t)$ должна быть монотонной по каждому инструменту. Траектория должна быть оптимальной в том смысле, что она минимизирует функционал 

$\lambda \int_{0}^{k} Var(w(t))\,dt + \sum_{j=1}^{k} TC(w_j - w{j-1})$

Транзакционные издержки в этом случае (в отличие от задачи 6) являются квадратичной функцией по каждому инструменту. 

Литературы по этой задаче в последние годы очень много, ищите по кодовымсловам Almgren - Chriss portfolio execution model.